In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

url = URL.create(
drivername="postgresql",
)

engine = create_engine(url)

# завантажити перелік діагнозів
sql_diagnos = \
"""
 SELECT key AS code, 
        value AS code_name,
        kwd_name AS kwd_system
   FROM core.dim_rpt_dictionaries,
LATERAL json_each_text("kwd_values"::json) json_each_text(key, value)  
  WHERE is_current = 'Y' 
    AND kwd_name = 'eHealth/ICD10_AM/condition_codes'
"""
df = pd.read_sql(sql_diagnos, con=engine)
df.to_csv("./data/diagnosis.csv", index=False, 
          header=False,
          sep='|')

# завантажити перелік інтервенцій
sql_intervents = \
"""
 WITH action as (
SELECT
       code,
       kwd_name
  FROM core.dim_rpt_services s
 WHERE s.is_current = 'Y' 
   AND s.is_active
 UNION ALL
SELECT
       code,
       kwd_name
  FROM core.dim_rpt_service_groups sg
 WHERE sg.is_current = 'Y' 
   AND sg.is_active)

SELECT *
  FROM action
"""
dff = pd.read_sql(sql_intervents, con=engine)
dff.to_csv("./data/intervents.csv", index=False, 
           header=False,
           sep='|')

# Завантажити підстави госпіталізації
sql_adm_source = \
"""
 SELECT
        json_each_text.key AS key,
        json_each_text.value
   FROM core.dim_rpt_dictionaries,
LATERAL json_each_text("kwd_values"::json) json_each_text(key, value)
  WHERE is_current = 'Y'
    AND kwd_name::text = 'eHealth/encounter_admit_source'::text
"""   
df_adm = pd.read_sql(sql_adm_source, con=engine)
df_adm.to_csv("./data/admission_source.csv", index=False, header=False)

# Завантажити результати госпіталізації
sql_dis_mode = \
"""
SELECT
        json_each_text.key AS key,
        json_each_text.value
   FROM core.dim_rpt_dictionaries,
LATERAL json_each_text("kwd_values"::json) json_each_text(key, value)
  WHERE is_current = 'Y'
    AND kwd_name::text = 'eHealth/encounter_discharge_disposition'::text
"""
df_dis = pd.read_sql(sql_dis_mode, con=engine)
df_dis.to_csv("./data/discharge_mode.csv", index=False, header=False)